In [13]:
import pandas as pd
import requests
import gzip
from tqdm import tqdm
import io
import xml.etree.ElementTree as ET
from tqdm import tqdm

# !pip install multiprocess
from multiprocess import Pool

In [2]:
# !pip install line_profiler
# %load_ext line_profiler

In [3]:
r = requests.get('https://apkpure.com/sitemap.xml').content

In [6]:
fp = io.StringIO(r.decode())
root = ET.parse(fp).getroot()
urls = [c[0].text for c in root]
urls = [u for u in urls if not ('default' in u or 'topics' in u or 'tag' in u or 'group' in u)]

In [7]:
def extract_apps(sitemap_url):
    try:
        r = requests.get(sitemap_url).content
    except:
        print('error', sitemap_url)
        return pd.DataFrame()
    
    fp = io.StringIO(gzip.decompress(r).decode())
    sitemap_root = ET.parse(fp).getroot()
    apps = list(sitemap_root)
    apps = [app for app in apps if 'image' in app[4].tag]
    
    df = pd.DataFrame({
        'loc': [a[0].text for a in apps],
        'lastmod': [a[1].text for a in apps],
        'changefreq': [a[2].text for a in apps],
        'priority': [a[3].text for a in apps],
        'image_loc': [a[4][0].text for a in apps],
        'name': [a[4][1].text for a in apps],
    })
    df.lastmod = pd.to_datetime(df.lastmod)
    df['sitemap_url'] = sitemap_url
    return df

In [16]:
with Pool(16) as p:
    dfs = list(tqdm(p.imap_unordered(extract_apps, urls), total=len(urls)))

print('ok')
metadata = pd.concat(dfs, ignore_index=True)

  6%|▋         | 494/7774 [00:36<10:56, 11.08it/s]

error https://apkpure.com/sitemaps/books_and_reference-304.xml.gz
error https://apkpure.com/sitemaps/books_and_reference-303.xml.gz
error https://apkpure.com/sitemaps/books_and_reference-302.xml.gz
error https://apkpure.com/sitemaps/books_and_reference-305.xml.gz
error https://apkpure.com/sitemaps/books_and_reference-306.xml.gz


Process ForkPoolWorker-88:
Traceback (most recent call last):
  File "/Users/user/anaconda3/envs/dsc180/lib/python3.8/site-packages/multiprocess/process.py", line 313, in _bootstrap
    self.run()
  File "/Users/user/anaconda3/envs/dsc180/lib/python3.8/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/user/anaconda3/envs/dsc180/lib/python3.8/site-packages/multiprocess/pool.py", line 131, in worker
    put((job, i, result))
  File "/Users/user/anaconda3/envs/dsc180/lib/python3.8/site-packages/multiprocess/queues.py", line 365, in put
    obj = _ForkingPickler.dumps(obj)
  File "/Users/user/anaconda3/envs/dsc180/lib/python3.8/site-packages/multiprocess/reduction.py", line 54, in dumps
    cls(buf, protocol, *args, **kwds).dump(obj)
Process ForkPoolWorker-82:
Traceback (most recent call last):
  File "/Users/user/anaconda3/envs/dsc180/lib/python3.8/site-packages/dill/_dill.py", line 445, in dump
    StockPickler.dump(self

KeyboardInterrupt: 

In [ ]:
# %lprun -f extract_apps [extract_apps(url) for url in urls[:20]]

In [17]:
metadata.shape

(199498, 7)

  6%|▋         | 494/7774 [00:49<10:56, 11.08it/s]